# Q-Table learning in Python
### Solving OpenAI gym's FrozenLake environment with a Q-table:

# The FrozenLake environment

### The FrozenLake environment
The environment is like a board game. It consists of a simple 4x4 grid, with each cell corresponding to a property of the surface of a frozen lake.<br>
The goal is to obtain a frisbee located on the lake, which is covered with holes.<br>
<br>
<table style="width:150px">
  <tr>
    <td style="text-align:center">S</td>
    <td style="text-align:center">F</td> 
    <td style="text-align:center">F</td>
    <td style="text-align:center">F</td>
  </tr>
  <tr>
    <td style="text-align:center">F</td>
    <td style="text-align:center">H</td> 
    <td style="text-align:center">F</td>
    <td style="text-align:center">H</td>
  </tr>
  <tr>
    <td style="text-align:center">F</td>
    <td style="text-align:center">F</td> 
    <td style="text-align:center">F</td>
    <td style="text-align:center">H</td>
  </tr>
  <tr>
    <td style="text-align:center">H</td>
    <td style="text-align:center">F</td> 
    <td style="text-align:center">F</td>
    <td style="text-align:center">G</td>
  </tr>
</table>
    
Where,<br>
S: starting point, safe<br>
F: frozen surface, safe<br>
H: hole, fall to your doom<br>
G: goal, where the frisbee is located<br>
<br>
A full description of the FrozenLake environment can be found here: https://gym.openai.com/envs/FrozenLake-v0/<br>
<br>
Using reinforcement learning, we intend to enable an agent to determine the optimal policy for navigating the lake and obtaining the frisbee.

# Reinforcement Learning – Theory

## Markov decision processes (MDPs)
<br>
A [Markov decision process](https://en.wikipedia.org/wiki/Markov_decision_process) (MDP), is a complex sounding mathematical term for what is essentially a flow chart. Each box is a state $s$, and each connecting line is the value of taking an action $a$.<br>
<br>
Below is a diagram that might represent a student MDP (sleep is the final state):
<br>
![title](markov.png)
<br>
We are in some state $s$, and in this state we can take an action $a$. When we take action $a$, we move from state $s$ to state $s$', where we must decide which new action, $a$', to take.<br>
<br>
The FrozenLake environment, and indeed any decision problem, can be represented as such an MDP.<br>
In this environment, we can be in any of the 16 cells, and in each of these there are 4 actions the agent can take: moving up, down, left or right. Having taken that action, we find ourselves in the next cell, and must make a decision again, whether to move up, down, left or right, and so on, until we either fall into a hole, or reach the frisbee.

## The action-value function (Q-value)
<br>
We need to quantify the value of taking a given action in a given state.<br>
The Q-value $Q(s,a)$ is a function which takes an action $a$ and a state $s$ as input, and outputs the "value" of taking that action in that state. (_i.e._ how desirable is it to move up, down, left or right in the current cell.)<br>
<br>

## The Q-Table

We can make a table of the Q-values for each action and state, called a Q-table.<br>
The table size is 16x4:<br>
16 observable states (as the environment is a 4x4 grid), and <br>
4 possible actions (move: up, down, left, or right).<br>
<br>
The states are linearised into the rows of the Q-table, while the columns define the actions.<br>
Each entry defines the reward for perfoming the action corresponding to the column, in the state corresponding to the row.<br>
<br>
So the Q-table looks like this:

\begin{matrix}
 & \textbf{L} & \textbf{D} & \textbf{R} & \textbf{U} \\
 \hline
 \textbf{S}: & x & x & x & x \\
 \textbf{F}: & x & x & x & x \\
 \textbf{F}: & x & x & x & x \\
 \textbf{F}: & x & x & x & x \\
 \textbf{F}: & x & x & x & x \\
 \textbf{H}: & x & x & x & x \\
 \textbf{F}: & x & x & x & x \\
 \textbf{H}: & x & x & x & x \\
 \textbf{F}: & x & x & x & x \\
 \textbf{F}: & x & x & x & x \\
 \textbf{F}: & x & x & x & x \\
 \textbf{H}: & x & x & x & x \\
 \textbf{H}: & x & x & x & x \\
 \textbf{F}: & x & x & x & x \\
 \textbf{F}: & x & x & x & x \\
 \textbf{G}: & x & x & x & x \\
\end{matrix}

See: https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py

## Optimal policy
<br>
We want to find the optimal policy, $\pi^*$, for solving the environment. If we followed this policy, we would take the _best_ action in every state we entered, _i.e._ the action which would give us the greatest reward. Let's call this optimal action $a^*$. We can define this with relation to the Q-value in the current state as:<br>
<br>
$$ a^* = \underset{a}{argmax} \left( Q(s,a) \right)$$
<br>
Here, $\underset{a}{argmax} \left( Q(s,a) \right)$ refers to the fact that we are choosing the _input argument_, $a$ (the action), which gives us the maximum Q-value – as opposed to $\max{ \left( Q(s,a) \right) }$ which would give us the maximum Q-value itself.<br>
<br>
However, we cannot solve our equation for the best action, $a^*$, unless we know the value of $Q(s,a)$. To do this, we will need the Bellman equation.<br>

## The Bellman equation
<br>
The Bellman equation allows us to find the value of $Q(s,a)$. We know that the value of taking action $a$ in state $s$ is at least the reward that we receive in $s$'. Therefore: <br>
<br>
$\quad Q(s,a) = r$<br>
<br>
However we know that the ultimate value of being in a state is not just the short-term rewards, but also the long term rewards. (In this environment, we have to traverse the frozen surface without receiving any reward, before receiving a reward when we reach the goal, several states later.) Therefore: <br>
<br>
$\quad Q(s,a) = r + r_{\text{future}}$<br>
<br>
But how do we define $r_{\text{future}} $  ? We know that when we are in the next state $s$' we can take a new action, $a$'. So if we want to maximise our reward, then $r_{\text{future}}$ is the _maximum_ Q-value from _all_ of the possible actions we can take in the next state, $s$'. So: <br>
<br>
$\quad r_{\text{future}} = \max{ \left( Q(s,a) \right)}$<br>
<br>
Substituting this into our previous equation:<br>
<br>
$$ Q(s,a) = r + \max{ \left( Q(s,a) \right)}$$<br>
<br>
This is the Bellman equation in a _deterministic_ setting with a _finite number of states_.<br>
<br>
The Bellman equation can therefore be stated in very simple terms: the value of an action is the _immediate_ reward from taking that action, plus the _delayed, future_ reward received from taking the _best_ action in the next state (and the next state, and the state after that... and so on).<br>

## Dynamic programming
<br>
It is clear that the Bellman equation is recursive – if the value of every state and action were known, this would enable us to calculate the _optimum_ decision path, starting at the final, goal state, and working backwards. In [operations research](https://en.wikipedia.org/wiki/Operations_research) (a branch of mathematics), this is called [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming). However in contrast to dynamic programming, in a reinforcement learning setting there are some important differences:<br>

## Discounted present value of future rewards
<br>
In reinforcement learning, there are several factors which could cause an unmodified Bellman equation to malfunction:<br>
<br>
Unlike dynamic programming situations, the FrozenLake environment is not _fully deterministic_. Our environment is stochastic (there is an element of randomness), so actions do no always consistently move us to the the same state they did in previous episodes. This means that we should decrease the value of our projected future rewards the further into the future we project, because it becomes decreasingly likely that we will obtain exactly those rewards, because the random elements are likely to interfere with our plans. We can therefore introduce a variable called the _discount factor_, $\gamma$, (where $\gamma < 1$), and update the Bellman equation as follows:<br>
<br>
$$Q(s,a) = r + \gamma \left( \max{ \left( Q(s,a) \right)} \right)$$
<br>
Another problem with future rewards might occur if our timeline is _inifite_, _i.e._ our only goal is to maximise reward, indefinitely. This infinite timeline would pose a mathematical problem for calculating the present value of the future rewards: inifity never ends. So our algorithm can never find a finite value for the future reward, because it can always calculate one step further, and then one step after that, and so on, forever. There are a few possible solutions to this problem; the one we will use for this Q-Table learning example is a modification of the Bellman equation.<br>
We can find a finite value in mathematics for an infinite sum, if as the sum tends to infinity, the sum <a href="https://en.wikipedia.org/wiki/Series_(mathematics)#Convergent_series">converges</a> to a single value. The discount factor enables us to do this.<br>
The best intuitive explanation for this is that when $\gamma = 1$, the growth of time is proportional to the "growth" of the rewards, and thus as time is infinite, the sum is infinite. However when $\gamma < 1$, the rewards shrink faster than time grows – as we get closer to infinity the value of the rewards gets smaller and smaller and tends towards zero, giving a finite, rather than infinite answer.<br>
<br>
Therefore, in technical jargon: the Q-value $Q(s,a)$ of performing an action $a$ in a state $s$ is the immediate reward $r$, plus the _discounted present value of future rewards under an optimal policy_: $\gamma \left( \max{ \left( Q(s,a) \right)} \right)$.

## The multi-armed bandit problem
<br>
Unlike dynamic programming, in reinforcement learning the value of every state and action is _unknown_ – we don't know beforehand the value of taking a new action, or the value of being in a new state – we can only calculate the value of our actions _after_ we have taken them, and we can only calculate the value of being in a state _after_ we have entered it. Because we cannot _calculate backwards_ from the goal state to the initial state, we must use the equation to _predict forwards_.<br>
<br>
Of course for this FrozenLake environment, we actually _could_ calculate the optimum path, because this task is simple – however once the environment becomes more complicated, there is a [combinatorial explosion](https://en.wikipedia.org/wiki/Combinatorial_explosion), and solving the problem analytically becomes computationally infeasible.<br>
<br>
This creates a new problem: we don't know anything about the Q-values in the initial state, so our initial Q-values for all actions are the same (_i.e._ zero). If the Q-values of all actions and states are identical, then the algorithm cannot choose a _maximum_ value.<br>
<br>
Moreover, assuming the agent overcame this somehow and found an initial decision path, then if the algorithm always chose the maximum reward, the agent would _always_ follow the first decision path it found that returned a non-zero reward, as this would always be the maximum expected future reward, because it would be the _only_ expected future reward! The agent would never explore any alternative policy, and therefore never learn the optimal policy.<br>
<br>
We can solve this by injecting an element of randomness, $\epsilon$, when the agent is selecting the action to perform. Updating our previous equation:<br>
<br>
$$a^* = \underset{a}{argmax} \left( Q(s,a) + \epsilon \right)$$
<br>
Often $\epsilon$ will start off high and decay towards zero in reinforcement learning implementations.

# Python implementation
<br>
We are now ready to implement the above theory in code:

### Initial python code

In [15]:
import gym
import numpy as np

### Load the environment

In [16]:
env = gym.make('FrozenLake-v0')

[2018-02-02 22:14:52,471] Making new env: FrozenLake-v0


### Initialise some variables for use in the Q-Table learning algorithm

In [21]:
# Initialize the Q-table as an (observation_space x action_space) matrix filled with zeros – (16x4)
Q = np.zeros([env.observation_space.n,env.action_space.n]) # Q-table matrix

# Set learning hyperparameters
lr = .8 # learning rate coefficient: this defines how quickly/slowly the agent learns. (We don't have to update the Q-table with the full value of the Bellman equation, it's better to learn slower.)
gamma = .95 # discount factor: this defines how short/long-term oriented the agent is
num_episodes = 2000 # number of times to run the simulation and update the Q-table

# create list to contain total reward per episode
rList = [] # list of rewards

### Implement Q-Table learning algorithm

In [22]:
for i in range(num_episodes): # run the simulation for num_episodes
    # Initialise; reset environment and get first new observation
    s = env.reset() # reset the gym environment and obtain the initial state
    rTotal = 0 # initialise a variable equal to the total sum of the rewards
    done = False # initialise boolean instructing whether the episode has terminated or not
    j = 0 # initialise number of steps taken before termination of the episode

    # The Q-Table learning algorithm
    while j < 99: # 99 is an arbitrarily large number for the maximum number steps to take
        j+=1 # increase the number of steps by one

        # Choose an action by greedily (with noise) picking from Q table
        '''
        np.argmax(Q[s,:]) chooses the action which corresponds to the maximum Q-value in the Q-table

        np.random.randn(1,env.action_space.n) generates the noise: an array of shape (1, action_space) – (1, 4), filled with
        random floats sampled from a univariate “normal” (Gaussian) distribution, of mean 0 and variance 1.

        Multiplying by (1./(i+1)) ensures that the noise decreases as the number of episodes increases – thus over time the
        agent tends to be more consistent and less random in its choice of actions.
        '''
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1))) # a* = argmax_a(Q(s,a) + ε)

        # Get new state and reward from environment
        s1,r,done,_ = env.step(a) # env.step(a) function returns the result of the action in 4 values: observation (object), reward (float), done (boolean), info (dict). See https://gym.openai.com/docs/ Observations section. We will ignore the info value.

        # Update Q-Table with new knowledge
        '''
        Q(s,a) = r + γ*max(Q(s',a')) <=  This is the Bellman equation:
                                         The immediate reward plus the discounted present value of future rewards.

        To update the Q-values:
            Find the difference between this new Q-value and the previous Q-value (= 0 if there's no change)
                Δ = (r + γ*max(Q(s',a'))) - Q(s,a)
            Modify this difference with a learning rate (α) to ensure the agent doesn't learn "too fast"
                αΔ = α[ r + γ*max(Q(s',a')) - Q(s,a) ]
            Update our Q-table values with the value we had before, plus the difference
                Q'(s,a) = Q(s,a) + αΔ = Q(s,a) + α[ r + *max(Q(s',a')) - Q(s,a) ]
        '''
        Q[s,a] = Q[s,a] + lr*(r + gamma*np.max(Q[s1,:]) - Q[s,a]) # Recompute Q-values using the Bellman equation
        rTotal += r # add reward obtained in this state to the total reward
        s = s1 # set the state to the next state
        if done == True:
            break # exit the while loop and begin a new episode if done

    rList.append(rTotal) # list of the total reward receieved per episode

## Results

In [23]:
'''Display the results of the episodes'''
# Print score over time
print("Score over time: " +  str(sum(rList)/num_episodes)) # prints the average reward receieved for all episodes (range = [0:1])

# Print the final Q-Table values
print("Final Q-Table Values")
print(Q) # Display the final Q-table

Score over time: 0.4835
Final Q-Table Values
[[1.17748057e-01 5.84151928e-03 3.20746392e-03 5.51370470e-03]
 [3.84409971e-04 2.13695442e-04 2.51525470e-04 8.52527960e-02]
 [8.07424413e-02 1.97037396e-03 2.40513153e-03 5.52945731e-03]
 [1.57941762e-04 2.88007266e-03 1.65438605e-03 6.43466736e-03]
 [1.81936659e-01 1.99682993e-03 1.31907099e-03 1.34859388e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.52709765e-04 1.06275324e-08 1.66052318e-02 1.53437412e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.41038289e-04 0.00000000e+00 5.76622652e-04 3.52013264e-01]
 [0.00000000e+00 4.45484509e-01 5.19149017e-04 2.05283335e-04]
 [1.81100035e-01 1.31809384e-03 1.74964532e-04 1.05503922e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.00119384e-04 0.00000000e+00 3.83287167e-01 9.16975882e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 8.76432234e-01]
 [0.000000

In [43]:
''''Print the final greedy path'''
s = env.reset() # reset the gym environment and obtain the initial state
done = False # initialise boolean instructing whether the episode has terminated or not
j = 0 # initialise number of steps taken before termination of the episode
path = [0]

# The Q-Table learning algorithm
while j < 99: # 99 is an arbitrarily large number of maximum steps to take
    j+=1 # increase the number of steps by one

    # Choose an action by greedily picking from Q table
    a = np.argmax(Q[s,:]) # a* = argmax_a(Q(s,a))

    # Get new state and reward from environment
    s1,r,done,_ = env.step(a) # env.step(a) function returns the result of the action in 4 values: observation (object), reward (float), done (boolean), info (dict). See https://gym.openai.com/docs/ Observations section. We will ignore the info value.
    s = s1 # set the state to the next state
    path.append(s)
    if done == True:
        break # exit the while loop and begin a new episode if done
print(path)

[0, 0, 4, 4, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 4, 8, 8, 9, 10, 14, 10, 9, 8, 4, 8, 8, 8, 9, 8, 4, 8, 4, 0, 0, 0, 0, 4, 4, 8, 8, 9, 13, 13, 14, 15]
